In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import json
from ufc_fight_agent.tool import *
from ufc_fight_agent.ufc_data import *
from ufc_fight_agent.agent import Agent

In [3]:
@tool
def get_upcoming_matchups(
    max_events: Optional[int] = 1,
    title_fights_only: bool = False,
    weight_class: Optional[str] = None,
):
    """
    Fetches upcoming UFC fights with comprehensive fighter data and statistics.

    Parameters:
    - max_events: Limits the number of upcoming events to return (default: 1, which returns only the next upcoming event). Only increase this value when the user explicitly requests information about multiple events.
    - title_fights_only: When True, only returns championship fights
    - weight_class: Filter for a specific weight class (e.g., "Flyweight", "Bantamweight", "Featherweight", "Lightweight", "Welterweight", "Middleweight", "Light Heavyweight", "Heavyweight", "Women's Strawweight", "Women's Flyweight", "Women's Bantamweight")
    """
    upcoming_events = []

    events = extract_events(max_events=max_events)
    if not events:
        return upcoming_events

    for event in events:
        fights = extract_fights(event.event_url)
        filtered_fights = []

        for fight in fights:
            if title_fights_only and not fight.title_fight:
                continue
            if weight_class and fight.weight_class.lower() != weight_class.lower():
                continue
            fighter_1_stats, fighter_2_stats = extract_matchup(fight.fight_url)
            fight.fighter_1_stats = fighter_1_stats
            fight.fighter_2_stats = fighter_2_stats
            filtered_fights.append(fight)

        if filtered_fights:
            event.fights = filtered_fights
            upcoming_events.append(event)

    return upcoming_events

In [4]:
print(json.dumps(get_upcoming_matchups.to_claude_format(), indent=2))

{
  "name": "get_upcoming_matchups",
  "description": "Fetches upcoming UFC fights with comprehensive fighter data and statistics.",
  "input_schema": {
    "type": "object",
    "properties": {
      "max_events": {
        "type": "integer",
        "description": "Limits the number of upcoming events to return (default: 1, which returns only the next upcoming event). Only increase this value when the user explicitly requests information about multiple events.",
        "default": 1,
        "optional": true
      },
      "title_fights_only": {
        "type": "boolean",
        "description": "When True, only returns championship fights",
        "default": false,
        "optional": true
      },
      "weight_class": {
        "type": "string",
        "description": "Filter for a specific weight class (e.g., \"Flyweight\", \"Bantamweight\", \"Featherweight\", \"Lightweight\", \"Welterweight\", \"Middleweight\", \"Light Heavyweight\", \"Heavyweight\", \"Women's Strawweight\", \"W

In [5]:
agent = Agent(tools=[get_upcoming_matchups])

In [6]:
final_response = agent.run(
    "I want to watch a exciting title fight in the upcoming UFC events",
    thinking_mode=True,
)

========== USER QUERY ==========================================================
I want to watch a exciting title fight in the upcoming UFC events

========== THINKING MODE =======================================================
Enabled

========== STEP 1 ==============================================================
Determining Tool Usage

========== THINKING PROCESS ====================================================
The user is interested in "exciting title fights in the upcoming UFC events". Let's analyze what parameters we need for the `get_upcoming_matchups` function:

1. `title_fights_only`: This should be set to `true` since the user specifically mentioned "title fight".
2. `max_events`: The user mentioned "upcoming UFC events" (plural), which suggests they want to see title fights beyond just the next event. Since title fights are relatively rare (usually 1-2 per event at most), I should set this to a higher value to ensure we capture enough upcoming title fights. A value of 